# Option Strategy Analyser

Used to analyse a option strategy. The strategy is loaded from a json file. The format of the json file is 

{
    "symbol": "NIFTY",
    "exchange": "NSE",
    "legs":[
        {
            "expiry": "26OCT2016",
            "strkePrice": "10200",
            "type" :"CE",
            "position" : "long",
            "quantity" : "75",
            "lots" : "1",
            "entry" : "50",
            "exit" : "0",
            "brokarage": "10",
            "optionChain": "NIFTY26OCT2017.json"
        }
    ]
}

In [219]:
from ipywidgets import *
from IPython.display import display
from IPython.html import widgets
import json

optionStragegyFile = widgets.Text(description="option strategy", width=200)
optionStragegyFile.value = "strategy.json"
display(optionStragegyFile)

Text(value=u'strategy.json', description=u'option strategy')

In [220]:
with open(optionStragegyFile.value) as json_data:
    optionStragegy = json.load(json_data)

In [221]:
optionChain = {}
for index in range(0, len(d['legs'])) :
    name =  d['legs'][index]['optionChain'].encode('ascii','ignore')
    with open(name) as json_data:
        optionChain[name] = json.load(json_data)

In [222]:
optionStragegy['legs'][0]

{u'brokarage': 10.0,
 u'entry': 10.1,
 u'exit': 0.0,
 u'expiry': u'26OCT2016',
 u'lots': 1,
 u'optionChain': u'NIFTY.26OCT2017.json',
 u'position': u'short',
 u'quantity': 500,
 u'strikePrice': 940,
 u'type': u'CE'}

In [223]:
optionChain

{'NIFTY.26OCT2017.json': {u'data': {u'10000.00': {u'Strike Price': u'10000.00',
    u'call': {u'AskPrice': u'42.40',
     u'AskQty': u'600',
     u'BidPrice': u'41.75',
     u'BidQty': u'750',
     u'Chng in OI': u'851,025',
     u'IV': u'9.29',
     u'LTP': u'42.05',
     u'Net Chng': u'-2.45',
     u'OI': u'4,065,900',
     u'Volume': u'82,365'},
    u'put': {u'AskPrice': u'256.00',
     u'AskQty': u'75',
     u'BidPrice': u'250.10',
     u'BidQty': u'150',
     u'Chng in OI': u'67,575',
     u'IV': u'15.21',
     u'LTP': u'254.10',
     u'Net Chng': u'-15.55',
     u'OI': u'1,999,875',
     u'Volume': u'17,982'}},
   u'10050.00': {u'Strike Price': u'10050.00',
    u'call': {u'AskPrice': u'32.75',
     u'AskQty': u'600',
     u'BidPrice': u'30.35',
     u'BidQty': u'750',
     u'Chng in OI': u'16,950',
     u'IV': u'9.44',
     u'LTP': u'31.50',
     u'Net Chng': u'-3.20',
     u'OI': u'98,400',
     u'Volume': u'1,337'},
    u'put': {u'AskPrice': u'306.55',
     u'AskQty': u'600',
 

In [224]:
lowestStrikePrice =  optionStragegy['legs'][0]["strikePrice"]
highestStrikePrice = optionStragegy['legs'][0]["strikePrice"]

#get the highest and lowest strike price
for index in range(1,len(optionStragegy['legs']) - 1) :
    if (lowestStrikePrice > optionStragegy['legs'][index]["strikePrice"]) :
        lowestStrikePrice = optionStragegy['legs'][index]["strikePrice"]
    if (highestStrikePrice < optionStragegy['legs'][index]["strikePrice"]) :
        highestStrikePrice = optionStragegy['legs'][index]["strikePrice"]
        

#calculate breakeng price per leg
def breakEvenPrice(strikePrice, entry, brokarage, quantity, cp):
    if (cp == "CE") :
        return strikePrice + entry + brokarage/quantity
    return strikePrice - entry - brokarage/quantity

for index in range(0,len(optionStragegy['legs'])) :
    optionStragegy['legs'][index]["breakEvenPrice"] = breakEvenPrice(optionStragegy['legs'][index]["strikePrice"],
                                                                     optionStragegy['legs'][index]["entry"],
                                                                     optionStragegy['legs'][index]["brokarage"],
                                                                     optionStragegy['legs'][index]["quantity"],
                                                                     optionStragegy['legs'][index]["type"])

#calculate per leg P/L per final price
import pylab as pl
priceWindow = 200
priceStep = 0.1
priceMin = lowestStrikePrice - priceWindow
priceMax = highestStrikePrice + priceWindow

def callLongPayout(strikePrice, currentPrice, entry, quantity, brokarage) :
    bep = breakEvenPrice(strikePrice, entry, brokarage,quantity, "CE")
    cost = (entry*quantity + brokarage) * -1
    if (currentPrice <= strikePrice) :
        return cost
    elif (currentPrice < bep) :
        return cost + (currentPrice - strikePrice)*quantity
    else:
        return (currentPrice - bep ) * quantity
    
def callShortPayout(strikePrice, currentPrice, entry, quantity, brokarage) :
    bep = breakEvenPrice(strikePrice, entry, brokarage,quantity, "CE")
    cost = (entry*quantity - brokarage)
    if (currentPrice <= strikePrice) :
        return cost
    elif (currentPrice < bep) :
        return cost - (currentPrice - strikePrice)*quantity
    else:
        return (bep - currentPrice) * quantity
    
def putLongPayout(strikePrice, currentPrice, entry, quantity, brokarage) :
    bep = breakEvenPrice(strikePrice, entry, brokarage,quantity, "PE")
    cost = (entry*quantity + brokarage) * -1
    if (currentPrice >= strikePrice) :
        return cost
    elif ( currentPrice > bep) :
        return cost - (currentPrice - strikePrice)*quantity
    else:
        return (bep - currentPrice  ) * quantity
    
def putShortPayout(strikePrice, currentPrice, entry, quantity, brokarage) :
    bep = breakEvenPrice(strikePrice, entry, brokarage,quantity, "PE")
    cost = (entry*quantity - brokarage)
    if (currentPrice >= strikePrice) :
        return cost
    elif (currentPrice > bep) :
        return cost + (currentPrice - strikePrice)*quantity
    else:
        return (currentPrice - bep) * quantity
    
payoutPerPrice = {
    "CE_long"  : callLongPayout,
    "CE_short" : callShortPayout,
    "PE_long"  : putLongPayout,
    "PE_short" : putShortPayout
}

for index in range(0,len(optionStragegy['legs'])) :
    optionStragegy['legs'][index]["payout"] = [] 
    for price in pl.frange(priceMin,priceMax,priceStep):
        key = optionStragegy['legs'][index]["type"] + "_" + optionStragegy['legs'][index]["position"]
        p = [price, payoutPerPrice[key](optionStragegy['legs'][index]["strikePrice"],
                                            price,
                                            optionStragegy['legs'][index]["entry"],
                                            optionStragegy['legs'][index]["quantity"],
                                            optionStragegy['legs'][index]["brokarage"])]
        optionStragegy['legs'][index]["payout"].append(p)

finalPayout =  {
    "positive" : [],
    "negative" : []
}


for index in range(0, len(optionStragegy['legs'][index]["payout"])) :
    p = -1
    t = 0
    for index1 in range(0,len(optionStragegy['legs'])) :
        if (p == -1 ) :
           p = optionStragegy['legs'][index1]["payout"][index][0]
        elif (p != optionStragegy['legs'][index1]["payout"][index][0]) :
            raise Exception("Price mismatch")
        t = t + optionStragegy['legs'][index1]["payout"][index][1]
    if (t >0) :
        finalPayout["positive"].append([p,t])
    else :
        finalPayout["negative"].append([p,t])

In [227]:
optionStragegy['legs'][0]["breakEvenPrice"] 

950.12

In [230]:
optionStragegy['legs'][1]["breakEvenPrice"] 

966.62

In [231]:
optionStragegy['legs'][2]["breakEvenPrice"] 

851.4300000000001

In [232]:
optionStragegy['legs'][3]["breakEvenPrice"] 

834.9300000000001

In [233]:
import charts

series = []

for index in range(0,len(optionStragegy['legs'])) :
    pseries = dict (
        type = 'line',
        name = str(optionStragegy['legs'][index]["strikePrice"]) + ":" +optionStragegy['legs'][index]["type"] + ":" + optionStragegy['legs'][index]["position"],
        data = optionStragegy['legs'][index]["payout"],
    )
    series.append(pseries)
    
tpseries = dict (
    type = "area",
    name = "FinalPayoutPositive",
    data = finalPayout["positive"]
)

tnseries = dict (
    type = "area",
    name = "FinalPayoutNegative",
    data = finalPayout["negative"]
)

series.append(tpseries)
series.append(tnseries)

options = dict(    
    title=dict(text= name + " Daily Historical"),
    chart=dict(zoomType='x'
               #, height= 500
              ),
    rangeSelector=dict( selected= 1)  ,
    tooltip=dict(split= True ),
)


charts.plot(series, show='inline', stock=False, options=options)